In [14]:
import numpy as np
import cv2

# Load the cascade classifiers from the default OpenCV data directory
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')

# Check if the cascades are loaded correctly
if face_cascade.empty() or eye_cascade.empty():
    raise IOError("Could not load cascade classifiers")

# Variable to store execution state
first_read = True

# Start the video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

ret, img = cap.read()
if not ret:
    raise IOError("Failed to read from webcam")

# Converting the recorded image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Applying filter to remove impurities
gray = cv2.bilateralFilter(gray, 5, 1, 1)

# Detecting the face for the region of the image to be fed to eye classifier
faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(200, 200))
if len(faces) > 0:
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # roi_face is face which is input to eye classifier
        roi_face = gray[y:y + h, x:x + w]
        roi_face_clr = img[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_face, 1.3, 5, minSize=(50, 50))

        # Examining the length of eyes object for eyes
        if len(eyes) >= 2:
            # Check if program is running for detection
            if first_read:
                cv2.putText(img, 
                            "Eye detected, press s to begin", 
                            (70, 70), 
                            cv2.FONT_HERSHEY_PLAIN, 3,
                            (0, 255, 0), 2)
            else:
                cv2.putText(img, 
                            "Eyes open!", (70, 70), 
                            cv2.FONT_HERSHEY_PLAIN, 2,
                            (255, 255, 255), 2)
        else:
            if first_read:
                # To ensure if the eyes are present before starting
                cv2.putText(img, 
                            "No eyes detected", (70, 70),
                            cv2.FONT_HERSHEY_PLAIN, 3,
                            (0, 0, 255), 2)
            else:
                # This will print on console and restart the algorithm
                print("Blink detected--------------")
                cv2.waitKey(3000)
                first_read = True
else:
    cv2.putText(img,
                "No face detected", (100, 100),
                cv2.FONT_HERSHEY_PLAIN, 3, 
                (0, 255, 0), 2)

# Display the image (commented out for headless environments)
# cv2.imshow('img', img)

# Save the image for review
cv2.imwrite('output.png', img)

# Release the camera
cap.release()
